In [1]:
import sys
import os
import glob
import pandas as pd
import numpy as np

In [2]:
#CONSTANTS
STOCK_PATH = "/home/alekzandr/Documents/robinhood/Daily_Stock_Report/Stocks/"

In [3]:
def get_ema(stock_file, history):
    column_name = 'ewm_' + str(history)
    finance_data = pd.read_csv(stock_file)
    finance_data[column_name] = finance_data['Close'].ewm(span=history,min_periods=0,adjust=False,ignore_na=False).mean()
    return finance_data

def get_sma(stock_file, history):
    column_name = 'sma' + str(history)
    finance_data = pd.read_csv(stock_file)
    finance_data[column_name] = finance_data['Close'].rolling(window=history).mean()
    return finance_data

def get_stock_list_path(stocks, path_to_stock_history):
    list_files = []
    for stock in stocks:
        list_files.append(path_to_stock_history + stock + ".csv")
    return list_files

def get_obv(list_files, history=20):

    # OBV Analysis, feel free to replace this section with your own analysis -------------------------------------------------------------------------
    new_data = [] #  This will be a 2D array to hold our stock name and OBV score
    interval = 0  # Used for iteration
    while interval < len(list_files):
        file = list_files[interval]
        Data = pd.read_csv(file).tail(history)  # Gets the last 10 days of trading for the current stock in iteration
        pos_move = []  # List of days that the stock price increased
        neg_move = []  # List of days that the stock price increased
        OBV_Value = 0  # Sets the initial OBV_Value to zero
        count = 0
        while (count < history):  # 20 because we are looking at the last 10 trading days
            if Data.iloc[count,1] < Data.iloc[count,4]:  # True if the stock increased in price
                pos_move.append(count)  # Add the day to the pos_move list
            elif Data.iloc[count,1] > Data.iloc[count,4]:  # True if the stock decreased in price
                neg_move.append(count)  # Add the day to the neg_move list
            count += 1
        count2 = 0
        for i in pos_move:  # Adds the volumes of positive days to OBV_Value, divide by opening price to normalize across all stocks
            OBV_Value = round(OBV_Value + (Data.iloc[i,5]/Data.iloc[i,1]))
        for i in neg_move:  # Subtracts the volumes of negative days from OBV_Value, divide by opening price to normalize across all stocks
            OBV_Value = round(OBV_Value - (Data.iloc[i,5]/Data.iloc[i,1]))
        Stock_Name = ((os.path.basename(list_files[interval])).split(".csv")[0])  # Get the name of the current stock we are analyzing
        new_data.append([Stock_Name, OBV_Value])  # Add the stock name and OBV value to the new_data list
        interval += 1
        
    return new_data

In [4]:
# Get a list of the Stocks
list_files = (glob.glob("/home/alekzandr/Documents/robinhood/Daily_Stock_Report/Stocks/*.csv"))
obv_data = get_obv(list_files)

In [5]:
obv_sorted = pd.DataFrame(obv_data, columns = ['Stock', 'OBV_Value'])  # Creates a new dataframe from the new_data list
obv_sorted["Stocks_Ranked"] = obv_sorted["OBV_Value"].rank(ascending = False)  # Rank the stocks by their OBV_Values
obv_sorted.sort_values("OBV_Value", inplace = True, ascending = False)  # Sort the ranked stocks

In [6]:
potential_stocks = obv_sorted["Stock"].head(100).tolist()
list_files = get_stock_list_path(potential_stocks, STOCK_PATH)

In [ ]:
list_files = (glob.glob(STOCK_PATH+"*.csv"))
i = 0
while (i < len(list_files)):
    
    # Get EWA for 5 Days
    data = get_ema(list_files[i], 5)
    try:
        data.drop('Unnamed: 0', axis=1, inplace=True)
    except:
        pass
    
    # Get EWA for 20 Days
    data.to_csv(list_files[i], index=False)
    
    data = get_ema(list_files[i], 20)
    try:
        data.drop('Unnamed: 0', axis=1, inplace=True)
    except:
        pass
    
    data.to_csv(list_files[i], index=False)
    
    # Get SMA for 5 Days
    data.to_csv(list_files[i], index=False)
    
    data = get_sma(list_files[i], 5)
    try:
        data.drop('Unnamed: 0', axis=1, inplace=True)
    except:
        pass
    
    data.to_csv(list_files[i], index=False)
    
    # Get SMA for 20 Days
    data.to_csv(list_files[i], index=False)
    data = get_sma(list_files[i], 20)
    try:
        data.drop('Unnamed: 0', axis=1, inplace=True)
    except:
        pass
    
    data.to_csv(list_files[i], index=False)
    
    i += 1

In [9]:
i = 0
temp_data = []
while i < len(list_files):
    # Read in CSV files
    file = list_files[i]
    stock_data = pd.read_csv(file)
    last_entry = stock_data.iloc[-1]
    
    # Get stock symbol
    stock_name = ((os.path.basename(list_files[i])).split(".csv")[0])
    
    temp_data.append([stock_name, stock_data.iloc[-1,0], stock_data.iloc[-1,4], 
                    stock_data.iloc[-1,8], stock_data.iloc[-1,9],
                     stock_data.iloc[-1,10], stock_data.iloc[-1,11]])
    i += 1
    
    
screened_stocks = pd.DataFrame(temp_data, columns=['stock','date', 'close', 'ewa_short', 'ewa_long', 'sma_short', 'sma_long'])

In [10]:
screened_ewa = screened_stocks[screened_stocks['ewa_short'] > screened_stocks['ewa_long']]
final_screened = screened_ewa[screened_ewa['sma_short'] > screened_ewa['sma_long']]

In [11]:
final_screened['ewa_change'] = (final_screened['ewa_short'] - final_screened['ewa_long']) / final_screened['ewa_long'] * 100
final_screened['sma_change'] = (final_screened['sma_short'] - final_screened['sma_long']) / final_screened['sma_long'] * 100

<ipython-input-11-c5d2cd54928e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_screened['ewa_change'] = (final_screened['ewa_short'] - final_screened['ewa_long']) / final_screened['ewa_long'] * 100
<ipython-input-11-c5d2cd54928e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_screened['sma_change'] = (final_screened['sma_short'] - final_screened['sma_long']) / final_screened['sma_long'] * 100


In [12]:
final_screened.head()

,stock,date,close,ewa_short,ewa_long,sma_short,sma_long,ewa_change,sma_change
0,SEEL,2021-04-01,5.08,4.794835,4.179787,4.714,4.04950,14.714826,16.409434
1,CLVS,2021-04-01,6.99,6.816792,6.549230,6.710,6.31450,4.085397,6.263363
2,JFIN,2021-04-01,8.47,8.569106,7.591086,8.418,7.08450,12.883800,18.822781
3,APTO,2021-04-01,6.00,5.880067,4.970512,5.998,4.57950,18.299022,30.974996
4,PRQR,2021-04-01,6.54,6.907800,6.194271,7.259,5.89525,11.519177,23.133032


In [13]:
final_screened.sort_values('sma_change', ascending = False, inplace=True)

<ipython-input-13-1e5c1ae8c8ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_screened.sort_values('sma_change', ascending = False, inplace=True)


In [14]:
print(final_screened.head(10))

   stock        date      close  ewa_short   ewa_long  sma_short  sma_long  \
58  FRST  2021-04-01  15.060000  10.552780   4.023287      9.030   2.41700   
3   APTO  2021-04-01   6.000000   5.880067   4.970512      5.998   4.57950   
4   PRQR  2021-04-01   6.540000   6.907800   6.194271      7.259   5.89525   
2   JFIN  2021-04-01   8.470000   8.569106   7.591086      8.418   7.08450   
79  AMTX  2021-04-01  26.150000  24.106142  20.318287     23.608  20.06550   
0   SEEL  2021-04-01   5.080000   4.794835   4.179787      4.714   4.04950   
14  BOWX  2021-04-01  12.930000  11.736512  10.769000     11.722  10.43575   
66  CLMT  2021-04-01   6.240000   6.032505   5.438154      6.018   5.38800   
12  FNKO  2021-04-01  20.959999  19.843013  17.833937     19.624  17.57250   
89   ADV  2021-04-01  12.010000  11.838294  11.035426     11.854  10.72100   

    ewa_change  sma_change  
58  162.292517  273.603641  
3    18.299022   30.974996  
4    11.519177   23.133032  
2    12.883800   18.82278